In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

base = '/mnt/data2/mxdi/archive/hf-mirror/open_llama_3b'
base_model = AutoModelForCausalLM.from_pretrained(base,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(base)
pth_path = '/mnt/data2/mxdi/archive/FastChat/checkpoints'

peft_model = PeftModel.from_pretrained(
    base_model,
    pth_path,
    torch_dtype=torch.float16,
)

model = peft_model.merge_and_unload()
save_path = '/mnt/data2/mxdi/archive/models/rank8openllama3b'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

[2023-12-20 13:22:37,037] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


('/mnt/data2/mxdi/archive/models/rank8openllama3b/tokenizer_config.json',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/special_tokens_map.json',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/tokenizer.model',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/added_tokens.json',
 '/mnt/data2/mxdi/archive/models/rank8openllama3b/tokenizer.json')

: 

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = '/mnt/data2/mxdi/archive/hf-mirror/llama-7b'
model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/mxd/miniconda3/envs/xeval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.20it/s]


In [2]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [3]:
torch.cuda.is_available() 

True

In [6]:
model.to('cpu')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=1, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=1, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dr

In [2]:
pth_path = '/mnt/data2/mxdi/archive/FastChat/checkpoints/lora_rank8_shuffle/checkpoint-100'
peft_model = PeftModel.from_pretrained(
    model,
    pth_path,
    torch_dtype=torch.float16,
)

只能说雀食很奇怪

* 加载model不用to是不会到device上的

* 但是cpu的model再使用peft.from_pretrained的时候是会把部分加载到device上的

* 但是device上的model再使用peft的时候，device上的存储占用是不会变化的

In [1]:
import transformers

model_name_or_path = '/mnt/data2/mxdi/archive/models/Llama-2-7b-hf'
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import torch

device = torch.device("cuda:0" )
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [7]:
next(model.parameters())

Parameter containing:
tensor([[ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
         -6.5565e-06,  8.9407e-07],
        [ 1.8616e-03, -3.3722e-03,  3.9864e-04,  ..., -8.3008e-03,
          2.5787e-03, -3.9368e-03],
        [ 1.0986e-02,  9.8877e-03, -5.0964e-03,  ...,  2.5177e-03,
          7.7057e-04, -5.0049e-03],
        ...,
        [-1.3977e-02, -2.7313e-03, -1.9897e-02,  ..., -1.0437e-02,
          9.5825e-03, -1.8005e-03],
        [-1.0742e-02,  9.3384e-03,  1.2939e-02,  ..., -3.3203e-02,
         -1.6357e-02,  3.3875e-03],
        [-8.3008e-03, -4.0588e-03, -1.1063e-03,  ...,  3.4790e-03,
         -1.2939e-02,  3.1948e-05]], device='cuda:0', requires_grad=True)

In [9]:
model.device.index

0

# 加载模型from pth

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import torch

device = torch.device('cpu')
base = '/home/mxd/archive/hf_mirror/llama-7b-hf'
base_model = AutoModelForCausalLM.from_pretrained(base,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(base)
pth_path = '/home/mxd/archive/FastChat/checkpoints/plora_r8_0121_interval400_1e-4_099_constant_4-6save/5046-savemodel.pth'
state_dict = torch.load(pth_path,map_location=device)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(base_model, lora_config)
rotary_flag = False
for keys in model.state_dict().keys():
    if 'rotary_emb' in keys:
        rotary_flag = True
        break
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if not rotary_flag:
        if 'rotary_emb' in k:
            continue

    if 'module.' in k:
        name = k[7:]
    else:
        name = k
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model = model.merge_and_unload()

save_path = '/home/mxd/archive/FastChat/checkpoints/plora_r8_0121_interval400_1e-4_099_constant_all_save_12615'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('/home/mxd/archive/FastChat/checkpoints/plora_r8_0121_interval400_1e-4_099_constant_all_save_10092/tokenizer_config.json',
 '/home/mxd/archive/FastChat/checkpoints/plora_r8_0121_interval400_1e-4_099_constant_all_save_10092/special_tokens_map.json',
 '/home/mxd/archive/FastChat/checkpoints/plora_r8_0121_interval400_1e-4_099_constant_all_save_10092/tokenizer.model',
 '/home/mxd/archive/FastChat/checkpoints/plora_r8_0121_interval400_1e-4_099_constant_all_save_10092/added_tokens.json',
 '/home/mxd/archive/FastChat/checkpoints/plora_r8_0121_interval400_1e-4_099_constant_all_save_10092/tokenizer.json')

In [7]:
model.state_dict().keys()

odict_keys(['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.rotary_emb.inv_freq', 'base

In [12]:
rotary_flag = False
for keys in model.state_dict().keys():
    if 'rotary_emb' in keys:
        rotary_flag = True
if rotary_flag:
    print('need to lap up rotary_emb')

base_model.model.model.layers.0.self_attn.rotary_emb.inv_freq


In [11]:
a

False

In [9]:
state_dict.keys()

odict_keys(['module.base_model.model.model.embed_tokens.weight', 'module.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'module.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'module.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'module.base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight', 'module.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'module.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'module.base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight', 'module.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'module.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'module.base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight', 'module.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'module.base_model.model.model.layers.0.self_attn.o_proj.lor

# 加载模型 from lora

In [1]:
lora_path = '/home/mxd/archive/FastChat/checkpoints/rank8_0118_constant_1e-4_099_6ep/checkpoint-5046'
base_path = '/home/mxd/archive/hf_mirror/llama-7b-hf' 

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
tokenizer = AutoTokenizer.from_pretrained(base_path)

model = AutoModelForCausalLM.from_pretrained(base_path)
model = PeftModel.from_pretrained(model, lora_path)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = model.merge_and_unload()

save_path = '/home/mxd/archive/FastChat/checkpoints/rank8_0118_constant_1e-4_099_6ep_checkpoint-5046'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/home/mxd/archive/FastChat/checkpoints/rank8_0118_constant_1e-4_099_6ep_checkpoint-5046/tokenizer_config.json',
 '/home/mxd/archive/FastChat/checkpoints/rank8_0118_constant_1e-4_099_6ep_checkpoint-5046/special_tokens_map.json',
 '/home/mxd/archive/FastChat/checkpoints/rank8_0118_constant_1e-4_099_6ep_checkpoint-5046/tokenizer.model',
 '/home/mxd/archive/FastChat/checkpoints/rank8_0118_constant_1e-4_099_6ep_checkpoint-5046/added_tokens.json',
 '/home/mxd/archive/FastChat/checkpoints/rank8_0118_constant_1e-4_099_6ep_checkpoint-5046/tokenizer.json')

# modelscope


In [1]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = 'f3699f4c-f2df-4f31-b390-4eb6ff3ecf17'

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="tanhe777/muti_llama", 
    model_dir="/home/mxd/archive/FastChat/checkpoints/plora_r8_0119_interval400_1e-4_099_constant_07lora_3save" # 本地模型目录，要求目录中必须包含configuration.json
)

2024-01-21 11:25:02,885 - modelscope - INFO - PyTorch version 2.1.2+cu118 Found.
2024-01-21 11:25:02,887 - modelscope - INFO - Loading ast index from /home/mxd/.cache/modelscope/ast_indexer
2024-01-21 11:25:02,922 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 d1a5118a57bce80c9af92a2fdd7e0f0f and a total number of 953 components indexed
2024-01-21 11:39:40,007 - modelscope - INFO - [master eb12996] 'upload model'
 13 files changed, 45552 insertions(+), 13 deletions(-)
 create mode 100644 2523-savemodel.pth
 create mode 100644 5046-savemodel.pth
 create mode 100644 7569-savemodel.pth
 delete mode 100644 README.md
 create mode 100644 config.json
 create mode 100644 generation_config.json
 create mode 100644 pytorch_model-00001-of-00002.bin
 create mode 100644 pytorch_model-00002-of-00002.bin
 create mode 100644 pytorch_model.bin.index.json
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer.model
 create mode 100644 tokenizer_conf